# Initial Setup


In [1]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install swissparlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 71 kB 3.3 MB/s 


In [4]:
import swissparlpy
import requests
import pandas as pd
import os
import urllib3
from datetime import datetime
import swissparlpy as spp
import datetime
from datetime import date
import numpy as np

In [5]:
urllib3.disable_warnings()
__location__ = os.path.realpath(os.getcwd())

In [6]:
# Create client with custom session
session = requests.Session()
session.verify = False # disable SSL verification
client = swissparlpy.SwissParlClient(session=session)

# List all available tables and properties


For information

In [ ]:
client.get_tables() # get list of all tables

['MemberParty',
 'Party',
 'Person',
 'PersonAddress',
 'PersonCommunication',
 'PersonInterest',
 'Session',
 'Committee',
 'MemberCommittee',
 'Canton',
 'Council',
 'Objective',
 'Resolution',
 'Publication',
 'External',
 'Meeting',
 'Subject',
 'Citizenship',
 'Preconsultation',
 'Bill',
 'BillLink',
 'BillStatus',
 'Business',
 'BusinessResponsibility',
 'BusinessRole',
 'LegislativePeriod',
 'MemberCouncil',
 'MemberParlGroup',
 'ParlGroup',
 'PersonOccupation',
 'RelatedBusiness',
 'BusinessStatus',
 'BusinessType',
 'MemberCouncilHistory',
 'MemberCommitteeHistory',
 'Vote',
 'Voting',
 'SubjectBusiness',
 'Transcript',
 'ParlGroupHistory',
 'Tags',
 'SeatOrganisationNr',
 'PersonEmployee',
 'Rapporteur',
 'Mutation']

In [ ]:
# print all tables with their properties

overview = swissparlpy.get_overview()
for table, props in overview.items():
    print(table)
    for prop in props:
        print(f' + {prop}')
    print('')

MemberParty
 + ID
 + Language
 + PartyNumber
 + PartyName
 + PersonNumber
 + PersonIdCode
 + FirstName
 + LastName
 + GenderAsString
 + PartyFunction
 + Modified
 + PartyAbbreviation

Party
 + ID
 + Language
 + PartyNumber
 + PartyName
 + StartDate
 + EndDate
 + Modified
 + PartyAbbreviation

Person
 + ID
 + Language
 + PersonNumber
 + PersonIdCode
 + Title
 + TitleText
 + LastName
 + GenderAsString
 + DateOfBirth
 + DateOfDeath
 + MaritalStatus
 + MaritalStatusText
 + PlaceOfBirthCity
 + PlaceOfBirthCanton
 + Modified
 + FirstName
 + OfficialName
 + MilitaryRank
 + MilitaryRankText
 + NativeLanguage
 + NumberOfChildren

PersonAddress
 + ID
 + Language
 + Modified
 + PersonNumber
 + AddressType
 + AddressTypeName
 + IsPublic
 + AddressLine1
 + AddressLine2
 + AddressLine3
 + City
 + CantonName
 + Comments
 + CantonNumber
 + Postcode
 + CantonAbbreviation

PersonCommunication
 + ID
 + Language
 + PersonNumber
 + Address
 + CommunicationType
 + CommunicationTypeText
 + Modified

PersonIn

# Table Voting

This script downloads votes from the Voting table by iterating over each session in a legislative period. The chunks are then saved in a directory as pickled DataFrames. We are using the legislative period 51 which containes the sessions we are interested in (5113, 5114, 5115, 5116)

! Takes 5 min to run !

In [7]:
path = os.path.join(__location__, "voting51")

def save_votes_of_session(id):
    if not os.path.exists(path):
        os.mkdir(path)
    pickle_path = os.path.join(path, f'{id}.pks')
    
    if os.path.exists(pickle_path):
        print(f"File {pickle_path} already exists, skipping")
        return
    
    print(f"Loading votes of session {id}...")
    data = client.get_data("Voting", Language="DE", IdSession=id)
    print(f"{data.count} rows loaded.")
    df = pd.DataFrame(data)
    
    df.to_pickle(pickle_path)
    print(f"Saved pickle at {pickle_path}")
    print("")


# get all session of the 51 legislative period
sessions51 = client.get_data("Session", Language="DE", LegislativePeriodNumber=51)
sessions51.count

for session in sessions51[12:16]:
    print(f"Loading session {session['ID']}")
    save_votes_of_session(session['ID'])

# Combine to one dataframe
path = os.path.join(__location__, "voting51")
df_voting51 = pd.concat([pd.read_pickle(os.path.join(path, x)) for x in os.listdir(path)])
df_voting51

#output after running

#Loading session 5113
#59000 rows loaded.
#Saved pickle at /content/voting51b/5113.pks
#Loading session 5115
#54400 rows loaded.
#Saved pickle at /content/voting51b/5115.pks
#Loading session 5114
#19200 rows loaded.
#Saved pickle at /content/voting51b/5114.pks
#Loading session 5116
#59000 rows loaded.
#Saved pickle at /content/voting51b/5116.pks

Loading session 5113
Loading votes of session 5113...
59000 rows loaded.
Saved pickle at /content/voting51/5113.pks

Loading session 5115
Loading votes of session 5115...
54400 rows loaded.
Saved pickle at /content/voting51/5115.pks

Loading session 5114
Loading votes of session 5114...
19200 rows loaded.
Saved pickle at /content/voting51/5114.pks

Loading session 5116
Loading votes of session 5116...
59000 rows loaded.
Saved pickle at /content/voting51/5116.pks



,ID,Language,IdVote,RegistrationNumber,PersonNumber,FirstName,LastName,Canton,CantonName,ParlGroupCode,...,BusinessTitle,BillTitle,IdLegislativePeriod,IdSession,VoteEnd,MeaningYes,MeaningNo,CantonID,Subject,VoteEndWithTimezone
0,4635320,DE,28693,24953,3923,Ada,Marra,VD,Waadt,S,...,Staatsrechnung 2021,Bundesbeschluss I über die Eidgenössische Staa...,51,5115,2022-05-30 16:10:12.783000+00:00,Antrag der Mehrheit und des Bundesrates,Antrag der Minderheit Andrey,23,Art. 2,2022-05-30 16:10:12.783000+00:00
1,4635321,DE,28693,24953,3897,Tiana Angelina,Moser,ZH,Zürich,GL,...,Staatsrechnung 2021,Bundesbeschluss I über die Eidgenössische Staa...,51,5115,2022-05-30 16:10:12.783000+00:00,Antrag der Mehrheit und des Bundesrates,Antrag der Minderheit Andrey,26,Art. 2,2022-05-30 16:10:12.783000+00:00
2,4635322,DE,28693,24953,3876,Yvette,Estermann,LU,Luzern,V,...,Staatsrechnung 2021,Bundesbeschluss I über die Eidgenössische Staa...,51,5115,2022-05-30 16:10:12.783000+00:00,Antrag der Mehrheit und des Bundesrates,Antrag der Minderheit Andrey,12,Art. 2,2022-05-30 16:10:12.783000+00:00
3,4635323,DE,28693,24953,3877,Doris,Fiala,ZH,Zürich,RL,...,Staatsrechnung 2021,Bundesbeschluss I über die Eidgenössische Staa...,51,5115,2022-05-30 16:10:12.783000+00:00,Antrag der Mehrheit und des Bundesrates,Antrag der Minderheit Andrey,26,Art. 2,2022-05-30 16:10:12.783000+00:00
4,4635324,DE,28693,24953,3880,Andrea Martina,Geissbühler,BE,Bern,V,...,Staatsrechnung 2021,Bundesbeschluss I über die Eidgenössische Staa...,51,5115,2022-05-30 16:10:12.783000+00:00,Antrag der Mehrheit und des Bundesrates,Antrag der Minderheit Andrey,4,Art. 2,2022-05-30 16:10:12.783000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,4633915,DE,28683,24943,4323,Céline,Weber,VD,Waadt,GL,...,"Soziales Unternehmertum, Mitarbeiterbestimmung...",,51,5114,2022-05-11 12:46:19.834000+00:00,Annahme des Postulates,Ablehnung des Postulats,23,,2022-05-11 12:46:19.837000+00:00
19196,4633916,DE,28683,24943,4324,Patricia,von Falkenstein,BS,Basel-Stadt,RL,...,"Soziales Unternehmertum, Mitarbeiterbestimmung...",,51,5114,2022-05-11 12:46:19.834000+00:00,Annahme des Postulates,Ablehnung des Postulats,6,,2022-05-11 12:46:19.837000+00:00
19197,4633917,DE,28683,24943,4326,Benjamin,Fischer,ZH,Zürich,V,...,"Soziales Unternehmertum, Mitarbeiterbestimmung...",,51,5114,2022-05-11 12:46:19.834000+00:00,Annahme des Postulates,Ablehnung des Postulats,26,,2022-05-11 12:46:19.837000+00:00
19198,4633918,DE,28683,24943,4327,Raphaël,Mahaim,VD,Waadt,G,...,"Soziales Unternehmertum, Mitarbeiterbestimmung...",,51,5114,2022-05-11 12:46:19.834000+00:00,Annahme des Postulates,Ablehnung des Postulats,23,,2022-05-11 12:46:19.837000+00:00


In [8]:
# checking table size, is there a limit of 59000 rows ?
df_5113 = pd.read_pickle(os.path.join(__location__, "voting51", '5113.pks'))
df_5113.shape

(59000, 27)

In [9]:
df_5114 = pd.read_pickle(os.path.join(__location__, "voting51", '5114.pks'))
df_5114.shape

(19200, 27)

In [10]:
df_5115 = pd.read_pickle(os.path.join(__location__, "voting51", '5115.pks'))
df_5115.shape

(54400, 27)

In [11]:
df_5116 = pd.read_pickle(os.path.join(__location__, "voting51", '5116.pks'))
df_5116.shape

(59000, 27)

In [12]:
df_5116.head(5) #preview

,ID,Language,IdVote,RegistrationNumber,PersonNumber,FirstName,LastName,Canton,CantonName,ParlGroupCode,...,BusinessTitle,BillTitle,IdLegislativePeriod,IdSession,VoteEnd,MeaningYes,MeaningNo,CantonID,Subject,VoteEndWithTimezone
0,4694120,DE,29059,25319,3923,Ada,Marra,VD,Waadt,S,...,Die Schweiz voranbringen. Digitale Leuchtturmp...,,51,5116,2022-09-12 15:04:50.025000+00:00,Antrag der Mehrheit (Ablehnung der Motion),Antrag der Minderheit Kutter (übernommen von S...,23,,2022-09-12 15:04:50.027000+00:00
1,4694121,DE,29059,25319,3897,Tiana Angelina,Moser,ZH,Zürich,GL,...,Die Schweiz voranbringen. Digitale Leuchtturmp...,,51,5116,2022-09-12 15:04:50.025000+00:00,Antrag der Mehrheit (Ablehnung der Motion),Antrag der Minderheit Kutter (übernommen von S...,26,,2022-09-12 15:04:50.027000+00:00
2,4694122,DE,29059,25319,3876,Yvette,Estermann,LU,Luzern,V,...,Die Schweiz voranbringen. Digitale Leuchtturmp...,,51,5116,2022-09-12 15:04:50.025000+00:00,Antrag der Mehrheit (Ablehnung der Motion),Antrag der Minderheit Kutter (übernommen von S...,12,,2022-09-12 15:04:50.027000+00:00
3,4694123,DE,29059,25319,3877,Doris,Fiala,ZH,Zürich,RL,...,Die Schweiz voranbringen. Digitale Leuchtturmp...,,51,5116,2022-09-12 15:04:50.025000+00:00,Antrag der Mehrheit (Ablehnung der Motion),Antrag der Minderheit Kutter (übernommen von S...,26,,2022-09-12 15:04:50.027000+00:00
4,4694124,DE,29059,25319,3880,Andrea Martina,Geissbühler,BE,Bern,V,...,Die Schweiz voranbringen. Digitale Leuchtturmp...,,51,5116,2022-09-12 15:04:50.025000+00:00,Antrag der Mehrheit (Ablehnung der Motion),Antrag der Minderheit Kutter (übernommen von S...,4,,2022-09-12 15:04:50.027000+00:00


In [13]:
# Keep only the columns we are interested in
df_51x = df_voting51[["IdVote", "PersonNumber", "Decision", "DecisionText", "BusinessNumber", "BusinessTitle", "IdSession"]]
df_51x

,IdVote,PersonNumber,Decision,DecisionText,BusinessNumber,BusinessTitle,IdSession
0,28693,3923,2,Nein,20220003,Staatsrechnung 2021,5115
1,28693,3897,1,Ja,20220003,Staatsrechnung 2021,5115
2,28693,3876,1,Ja,20220003,Staatsrechnung 2021,5115
3,28693,3877,1,Ja,20220003,Staatsrechnung 2021,5115
4,28693,3880,1,Ja,20220003,Staatsrechnung 2021,5115
...,...,...,...,...,...,...,...
19195,28683,4323,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19196,28683,4324,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19197,28683,4326,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19198,28683,4327,1,Ja,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114


In [14]:
# filter the sessions we are interested in: 5113, 5114, 5115, 5116  #this step should normally not be necessary but I loaded others (irrelevant) sessions while playing with the code
df_51_final = df_51x[df_51x["IdSession"] >= 5113]
df_51_final

,IdVote,PersonNumber,Decision,DecisionText,BusinessNumber,BusinessTitle,IdSession
0,28693,3923,2,Nein,20220003,Staatsrechnung 2021,5115
1,28693,3897,1,Ja,20220003,Staatsrechnung 2021,5115
2,28693,3876,1,Ja,20220003,Staatsrechnung 2021,5115
3,28693,3877,1,Ja,20220003,Staatsrechnung 2021,5115
4,28693,3880,1,Ja,20220003,Staatsrechnung 2021,5115
...,...,...,...,...,...,...,...
19195,28683,4323,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19196,28683,4324,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19197,28683,4326,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19198,28683,4327,1,Ja,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114


In [15]:
# We only keep DecisionText = Ja (1), Nein (2), Enthaltung (3)

voting_final = df_51_final[df_51_final.Decision < 4]
voting_final


,IdVote,PersonNumber,Decision,DecisionText,BusinessNumber,BusinessTitle,IdSession
0,28693,3923,2,Nein,20220003,Staatsrechnung 2021,5115
1,28693,3897,1,Ja,20220003,Staatsrechnung 2021,5115
2,28693,3876,1,Ja,20220003,Staatsrechnung 2021,5115
3,28693,3877,1,Ja,20220003,Staatsrechnung 2021,5115
4,28693,3880,1,Ja,20220003,Staatsrechnung 2021,5115
...,...,...,...,...,...,...,...
19195,28683,4323,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19196,28683,4324,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19197,28683,4326,2,Nein,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114
19198,28683,4327,1,Ja,20203559,"Soziales Unternehmertum, Mitarbeiterbestimmung...",5114


In [88]:
#Mount Drive to export data
#from google.colab import drive
#drive.mount('drive')

In [17]:
# df_51_final.to_csv('voting2022.csv', encoding="utf-8-sig") #generate backup csv

In [18]:
# Export csv to Drive
# !cp voting2022.csv "drive/My Drive/"

## Table Voting (other method, ignore)


Below code was an attempt to import data differently to see if the data size would be different (than 59'000) but it was not

In [19]:
# client.get_variables('Voting') # get list of variables of table 'Voting'

In [20]:
# creating a dataframe with data of session 5113
# voting = client.get_data('Voting', Language='DE', IdSession=5113) 
# voting_df = pd.DataFrame(voting)                                                                        

In [21]:
# voting_df.shape # 59'000 rows too

In [22]:
# filtering the colums we are interested in
# voting_dfx = voting_df [["PersonNumber", "Decision", "DecisionText", "BusinessNumber"]]
# voting_dfx

**Other method on Table Voting STOPS here**

# Table Member Council

In [23]:
client.get_variables('MemberCouncil') # get list of variables of table 'MemberCouncil'

['ID',
 'Language',
 'IdPredecessor',
 'PersonNumber',
 'PersonIdCode',
 'Active',
 'FirstName',
 'LastName',
 'GenderAsString',
 'Canton',
 'CantonName',
 'CantonAbbreviation',
 'Council',
 'CouncilName',
 'CouncilAbbreviation',
 'ParlGroupNumber',
 'ParlGroupName',
 'ParlGroupAbbreviation',
 'ParlGroupFunction',
 'ParlGroupFunctionText',
 'Party',
 'PartyName',
 'PartyAbbreviation',
 'MilitaryRank',
 'MilitaryRankText',
 'MaritalStatus',
 'MaritalStatusText',
 'Nationality',
 'BirthPlace_City',
 'BirthPlace_Canton',
 'Mandates',
 'AdditionalMandate',
 'AdditionalActivity',
 'OfficialName',
 'DateJoining',
 'DateLeaving',
 'DateElection',
 'DateOath',
 'DateResignation',
 'Modified',
 'NumberOfChildren',
 'Citizenship',
 'DateOfBirth',
 'DateOfDeath']

In [24]:
members = client.get_data('MemberCouncil', Language='DE')
members_df = pd.DataFrame(members)

In [25]:
#Importing member data with filter on Date Leaving (no date or after 01.01.2022) in order to keep only those active in 2022

members = spp.get_data(table="MemberCouncil", filter="(DateLeaving eq null or DateLeaving gt datetime'2022-01-01T00:00:00') and Language eq 'DE'")
members_df = pd.DataFrame(members)
members_df.shape

(258, 44)

In [26]:
# Keep only the columns we are interested in
members_dfx = members_df [["PersonNumber", "FirstName", "LastName", "GenderAsString", "CantonAbbreviation", "PartyName", "ParlGroupName","ParlGroupAbbreviation", "BirthPlace_Canton", "DateJoining", "DateLeaving", "DateOfBirth"]]
members_dfx

,PersonNumber,FirstName,LastName,GenderAsString,CantonAbbreviation,PartyName,ParlGroupName,ParlGroupAbbreviation,BirthPlace_Canton,DateJoining,DateLeaving,DateOfBirth
0,146,Ueli,Maurer,m,ZH,Schweizerische Volkspartei,,,ZH,2020-01-01 00:00:00+00:00,1753-01-01 00:00:00+00:00,1950-12-01 00:00:00+00:00
1,172,Paul,Rechsteiner,m,SG,Sozialdemokratische Partei der Schweiz,Sozialdemokratische Fraktion,S,SG,2019-12-02 00:00:00+00:00,2022-12-16 00:00:00+00:00,1952-08-26 00:00:00+00:00
2,491,Pierre-Yves,Maillard,m,VD,Sozialdemokratische Partei der Schweiz,Sozialdemokratische Fraktion,S,VD,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1968-03-16 00:00:00+00:00
3,512,Simonetta,Sommaruga,f,BE,Sozialdemokratische Partei der Schweiz,,,ZG,2021-01-01 00:00:00+00:00,1753-01-01 00:00:00+00:00,1960-05-14 00:00:00+00:00
4,525,Roberto,Zanetti,m,SO,Sozialdemokratische Partei der Schweiz,Sozialdemokratische Fraktion,S,SO,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1954-12-14 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
253,4326,Benjamin,Fischer,m,ZH,Schweizerische Volkspartei,Fraktion der Schweizerischen Volkspartei,V,ZH,2022-02-28 00:00:00+00:00,1753-01-01 00:00:00+00:00,1991-08-16 00:00:00+00:00
254,4327,Raphaël,Mahaim,m,VD,GRÜNE Schweiz,Grüne Fraktion,G,VD,2022-02-28 00:00:00+00:00,1753-01-01 00:00:00+00:00,1983-12-31 00:00:00+00:00
255,4328,Natalie,Imboden,f,BE,GRÜNE Schweiz,Grüne Fraktion,G,,2022-05-10 00:00:00+00:00,1753-01-01 00:00:00+00:00,1970-09-04 00:00:00+00:00
256,4329,Daniel,Ruch,m,VD,FDP.Die Liberalen,FDP-Liberale Fraktion,RL,VD,2022-06-13 00:00:00+00:00,1753-01-01 00:00:00+00:00,1963-03-20 00:00:00+00:00


In [27]:
# Add Age column in table
pd.options.mode.chained_assignment = None       #Deactivate SettingWithCopyWarning message
members_dfx[["DateOfBirth"]] = members_dfx[["DateOfBirth"]].apply(pd.to_datetime)
members_dfx['Age'] = date.today().year-members_dfx['DateOfBirth'].dt.year
members_dfx.head(5)

,PersonNumber,FirstName,LastName,GenderAsString,CantonAbbreviation,PartyName,ParlGroupName,ParlGroupAbbreviation,BirthPlace_Canton,DateJoining,DateLeaving,DateOfBirth,Age
0,146,Ueli,Maurer,m,ZH,Schweizerische Volkspartei,,,ZH,2020-01-01 00:00:00+00:00,1753-01-01 00:00:00+00:00,1950-12-01 00:00:00+00:00,72
1,172,Paul,Rechsteiner,m,SG,Sozialdemokratische Partei der Schweiz,Sozialdemokratische Fraktion,S,SG,2019-12-02 00:00:00+00:00,2022-12-16 00:00:00+00:00,1952-08-26 00:00:00+00:00,70
2,491,Pierre-Yves,Maillard,m,VD,Sozialdemokratische Partei der Schweiz,Sozialdemokratische Fraktion,S,VD,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1968-03-16 00:00:00+00:00,54
3,512,Simonetta,Sommaruga,f,BE,Sozialdemokratische Partei der Schweiz,,,ZG,2021-01-01 00:00:00+00:00,1753-01-01 00:00:00+00:00,1960-05-14 00:00:00+00:00,62
4,525,Roberto,Zanetti,m,SO,Sozialdemokratische Partei der Schweiz,Sozialdemokratische Fraktion,S,SO,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1954-12-14 00:00:00+00:00,68


In [28]:
# Add Seniority column in table (ancienneté) # Not possible here as DateJoining = Start date of current legislative period


In [29]:
# Creating a table with cantons and regions. We want to add a column about the language region in the dataframe Members Council

cantons_list = {'CantonAbbreviation': ['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH'],
        'Region': ['CH-de', 'CH-de', 'CH-de', 'CH-de', 'CH-de', 'CH-de', 'CH-fr', 'CH-fr', 'CH-de', 'CH-de', 'CH-fr', 'CH-de', 'CH-fr', 'CH-de', 'CH-de', 'CH-de', 'CH-de', 'CH-de', 'CH-de', 'CH-de', 'CH-it', 'CH-de', 'CH-fr', 'CH-fr', 'CH-de', 'CH-de']}
df_cantons = pd.DataFrame(cantons_list)
df_cantons

,CantonAbbreviation,Region
0,AG,CH-de
1,AI,CH-de
2,AR,CH-de
3,BE,CH-de
4,BL,CH-de
5,BS,CH-de
6,FR,CH-fr
7,GE,CH-fr
8,GL,CH-de
9,GR,CH-de


In [30]:
# Alternative: import cantons.csv from Drive
# from google.colab import files
# uploaded = files.upload()

In [31]:
# df_cantons = pd.read_csv('cantons.csv', sep =';')
# df_cantons

In [32]:
# Merge tables Members and Cantons (=add language region)
df_merge0 =  pd.merge(members_dfx, df_cantons, on='CantonAbbreviation', how='inner')
df_merge0

,PersonNumber,FirstName,LastName,GenderAsString,CantonAbbreviation,PartyName,ParlGroupName,ParlGroupAbbreviation,BirthPlace_Canton,DateJoining,DateLeaving,DateOfBirth,Age,Region
0,146,Ueli,Maurer,m,ZH,Schweizerische Volkspartei,,,ZH,2020-01-01 00:00:00+00:00,1753-01-01 00:00:00+00:00,1950-12-01 00:00:00+00:00,72,CH-de
1,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1964-06-03 00:00:00+00:00,58,CH-de
2,1153,Ruedi,Noser,m,ZH,FDP.Die Liberalen,FDP-Liberale Fraktion,RL,GL,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1961-04-14 00:00:00+00:00,61,CH-de
3,3877,Doris,Fiala,f,ZH,FDP.Die Liberalen,FDP-Liberale Fraktion,RL,ZH,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1957-01-29 00:00:00+00:00,65,CH-de
4,3882,Bastien,Girod,m,ZH,GRÜNE Schweiz,Grüne Fraktion,G,GE,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1980-12-21 00:00:00+00:00,42,CH-de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,4254,Damien,Cottier,m,NE,FDP.Die Liberalen,FDP-Liberale Fraktion,RL,NE,2022-02-28 00:00:00+00:00,1753-01-01 00:00:00+00:00,1975-04-03 00:00:00+00:00,47,CH-fr
254,4260,Fabien,Fivaz,m,NE,GRÜNE Schweiz,Grüne Fraktion,G,NE,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1978-02-25 00:00:00+00:00,44,CH-fr
255,4270,Baptiste,Hurni,m,NE,Sozialdemokratische Partei der Schweiz,Sozialdemokratische Fraktion,S,NE,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1986-04-04 00:00:00+00:00,36,CH-fr
256,4206,Erich,Ettlin,m,OW,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,OW,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1962-05-30 00:00:00+00:00,60,CH-de


In [33]:
#df_merge0.to_csv('members2022.csv', encoding="utf-8-sig") #generate backup csv

In [34]:
# Finally we want to do some party grouping
# Left = Social Democrats (S), Green Group (G)
# Center = Center Group (M-E)
# Right = Green Liberal Group (GL), Liberals Group (RL), Swiss People's Party (V)

# Create a table with party and their group

PartyGroup_list = {'ParlGroupAbbreviation': ['G', 'GL', 'M-E', 'RL', 'S', 'V'],
        'Region': ['Left', 'Right', 'Center', 'Right', 'Left', 'Right']}
df_party = pd.DataFrame(PartyGroup_list)
df_party

,ParlGroupAbbreviation,Region
0,G,Left
1,GL,Right
2,M-E,Center
3,RL,Right
4,S,Left
5,V,Right


In [35]:
# Add PartyGroup to the Member Council table. 
# The 7 Federal Councillors + the Chancellor don't belong to a ParlGroup so they will be deleted with the merging (-8), which is OK as they are not relevant for our project.

df_members_final =  pd.merge(df_merge0, df_party, on='ParlGroupAbbreviation', how='inner')
df_members_final

,PersonNumber,FirstName,LastName,GenderAsString,CantonAbbreviation,PartyName,ParlGroupName,ParlGroupAbbreviation,BirthPlace_Canton,DateJoining,DateLeaving,DateOfBirth,Age,Region_x,Region_y
0,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1964-06-03 00:00:00+00:00,58,CH-de,Right
1,3897,Tiana Angelina,Moser,f,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1979-04-06 00:00:00+00:00,43,CH-de,Right
2,4248,Judith,Bellaiche,f,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1971-02-02 00:00:00+00:00,51,CH-de,Right
3,4267,Corina,Gredig,f,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1987-09-08 00:00:00+00:00,35,CH-de,Right
4,4274,Jörg,Mäder,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1975-07-20 00:00:00+00:00,47,CH-de,Right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,4087,Jean-Paul,Gschwind,m,JU,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,JU,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1952-10-31 00:00:00+00:00,70,CH-fr,Center
246,4240,Charles,Juillard,m,JU,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,JU,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1962-12-17 00:00:00+00:00,60,CH-fr,Center
247,4243,Heidi,Z'graggen,f,UR,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,UR,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1966-02-01 00:00:00+00:00,56,CH-de,Center
248,4292,Simon,Stadler,m,UR,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,UR,2019-12-02 00:00:00+00:00,1753-01-01 00:00:00+00:00,1988-05-02 00:00:00+00:00,34,CH-de,Center


# Table Business


In [37]:
client.get_variables('Business') # get list of variables of table 'Business'

['ID',
 'Language',
 'BusinessShortNumber',
 'BusinessType',
 'BusinessTypeName',
 'BusinessTypeAbbreviation',
 'Title',
 'Description',
 'InitialSituation',
 'Proceedings',
 'DraftText',
 'SubmittedText',
 'ReasonText',
 'DocumentationText',
 'MotionText',
 'FederalCouncilResponseText',
 'FederalCouncilProposal',
 'FederalCouncilProposalText',
 'FederalCouncilProposalDate',
 'SubmittedBy',
 'BusinessStatus',
 'BusinessStatusText',
 'BusinessStatusDate',
 'ResponsibleDepartment',
 'ResponsibleDepartmentName',
 'ResponsibleDepartmentAbbreviation',
 'IsLeadingDepartment',
 'Tags',
 'Category',
 'Modified',
 'SubmissionDate',
 'SubmissionCouncil',
 'SubmissionCouncilName',
 'SubmissionCouncilAbbreviation',
 'SubmissionSession',
 'SubmissionLegislativePeriod',
 'FirstCouncil1',
 'FirstCouncil1Name',
 'FirstCouncil1Abbreviation',
 'FirstCouncil2',
 'FirstCouncil2Name',
 'FirstCouncil2Abbreviation',
 'TagNames']

In [38]:
# Not using this code as the output in df is limited to 1000 rows

#business = client.get_data('Business', Language='DE')
#print (business.count) 
# business_df = pd.DataFrame(business) 
# business_df

In [39]:
#business = spp.get_data(table="Business", filter="(SubmissionDate gt datetime'2022-01-01T00:00:00') and Language eq 'DE' and Sessions/ID eq# 5113")  # playing with time filter
#business = spp.get_data(table="Business", filter="(Sessions/ID eq 5113 or Sessions/ID eq 5114 or Sessions/ID eq 5115 or Sessions/ID eq 5116) and Language eq 'DE'")  # Limit of 1000 rows here too

#Split data by session to overcome 1000 rows limit
business1 = spp.get_data(table="Business", filter="Sessions/ID eq 5113 and Language eq 'DE'")
business2 = spp.get_data(table="Business", filter="Sessions/ID eq 5114 and Language eq 'DE'")
business3 = spp.get_data(table="Business", filter="Sessions/ID eq 5115 and Language eq 'DE'")
business4 = spp.get_data(table="Business", filter="Sessions/ID eq 5116 and Language eq 'DE'")

In [40]:
print (business1.count, business2.count, business3.count, business4.count) #check data size

769 176 654 762


In [41]:
#store each data lot in a dataframe
df_b1 = pd.DataFrame(business1)
df_b2 = pd.DataFrame(business2)
df_b3 = pd.DataFrame(business3)
df_b4 = pd.DataFrame(business4)

In [42]:
# group the dataframes of each sessions in one dataframe
df_business = pd.concat([df_b1, df_b2, df_b3, df_b4], axis=0)
df_business.shape

(2361, 43)

In [43]:
df_business.head(5)

,ID,Language,BusinessShortNumber,BusinessType,BusinessTypeName,BusinessTypeAbbreviation,Title,Description,InitialSituation,Proceedings,...,SubmissionCouncilAbbreviation,SubmissionSession,SubmissionLegislativePeriod,FirstCouncil1,FirstCouncil1Name,FirstCouncil1Abbreviation,FirstCouncil2,FirstCouncil2Name,FirstCouncil2Abbreviation,TagNames
0,20210081,DE,21.081,1,Geschäft des Bundesrates,BRG,Assistenzdienst der Armee zur Unterstützung de...,Botschaft vom 16. Februar 2022 zum Bundesbesch...,<text><p><b>Medienmitteilung des Bundesrates v...,<text><p>SDA-Meldung</p><p><b>Debatte im Ständ...,...,,5113,51,2,Ständerat,SR,1,Nationalrat,NR,Sicherheitspolitik|Gesundheit
1,20220001,DE,22.001,1,Geschäft des Bundesrates,BRG,Geschäftsbericht des Bundesrates 2021,Bericht des Bundesrates über seine Geschäftsfü...,<text><p><b>Medienmitteilung des Bundesrates v...,<text><p>SDA-Meldung</p><p><b>Debatte im Natio...,...,,5113,51,1,Nationalrat,NR,2,Ständerat,SR,Staatspolitik
2,20220005,DE,22.005,1,Geschäft des Bundesrates,BRG,Armeebotschaft 2022,Armeebotschaft 2022 vom 16. Februar 2022,<text><p><b>Medienmitteilung des Bundesrates v...,<text><p>SDA-Meldung</p><p><b>Debatte im Ständ...,...,,5113,51,2,Ständerat,SR,0,,,Sicherheitspolitik
3,20220006,DE,22.006,1,Geschäft des Bundesrates,BRG,Motionen und Postulate der gesetzgebenden Räte...,Bericht des Bundesrates vom 4. März 2022 über ...,<text><p><b>Bericht des Bundesrates vom 4. Mär...,<text><p><b>Auskünfte</b></p><p>Die zuständige...,...,,5113,51,1,Nationalrat,NR,2,Ständerat,SR,Parlament
4,20220007,DE,22.007,1,Geschäft des Bundesrates,BRG,Voranschlag 2022. Nachtrag I,Botschaft vom 2. Februar 2022 über den Nachtra...,<text><p><b>Medienmitteilung des Bundesrates v...,<text><p><b>Entwurf 1 und 2</b></p><p> </p><p>...,...,,5113,51,1,Nationalrat,NR,2,Ständerat,SR,Finanzwesen


In [44]:
# Keep only the columns we are interested in
business_dfx = df_business[["ID", "BusinessType", "BusinessTypeName", "ResponsibleDepartmentAbbreviation", "SubmissionSession", "TagNames"]]
business_dfx.head(10)

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,TagNames
0,20210081,1,Geschäft des Bundesrates,VBS,5113,Sicherheitspolitik|Gesundheit
1,20220001,1,Geschäft des Bundesrates,EDA,5113,Staatspolitik
2,20220005,1,Geschäft des Bundesrates,VBS,5113,Sicherheitspolitik
3,20220006,1,Geschäft des Bundesrates,BK,5113,Parlament
4,20220007,1,Geschäft des Bundesrates,EFD,5113,Finanzwesen
5,20220008,1,Geschäft des Bundesrates,WBF,5113,Internationale Politik|Wirtschaft|Finanzwesen
6,20220010,2,Geschäft des Parlaments,Parl,5113,
7,20220012,2,Geschäft des Parlaments,Parl,5113,
8,20220013,2,Geschäft des Parlaments,Parl,5113,
9,20220015,2,Geschäft des Parlaments,Parl,5113,


In [45]:
#business_dfx.to_csv('business2022.csv', encoding="utf-8-sig") #generate backup csv

In [46]:
# Split the tags in separate columns. A business item has max 8 tags in our dataset

business_dfx [['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5', 'Tag6', 'Tag7', 'Tag8']] = business_dfx ['TagNames'].str.split('|', 7, expand=True)
business_dfx.head()

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,TagNames,Tag1,Tag2,Tag3,Tag4,Tag5,Tag6,Tag7,Tag8
0,20210081,1,Geschäft des Bundesrates,VBS,5113,Sicherheitspolitik|Gesundheit,Sicherheitspolitik,Gesundheit,None,None,None,None,None,None
1,20220001,1,Geschäft des Bundesrates,EDA,5113,Staatspolitik,Staatspolitik,None,None,None,None,None,None,None
2,20220005,1,Geschäft des Bundesrates,VBS,5113,Sicherheitspolitik,Sicherheitspolitik,None,None,None,None,None,None,None
3,20220006,1,Geschäft des Bundesrates,BK,5113,Parlament,Parlament,None,None,None,None,None,None,None
4,20220007,1,Geschäft des Bundesrates,EFD,5113,Finanzwesen,Finanzwesen,None,None,None,None,None,None,None


In [47]:
# Unpivot Tag columns --> duplicate rows which have more than 1 tag, all tags in 1 column

business_dfx2 = pd.melt(business_dfx, id_vars= ['ID', 'BusinessType', 'BusinessTypeName', 'ResponsibleDepartmentAbbreviation', 'SubmissionSession'], value_vars=['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5'])
business_dfx2

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,variable,value
0,20210081,1,Geschäft des Bundesrates,VBS,5113,Tag1,Sicherheitspolitik
1,20220001,1,Geschäft des Bundesrates,EDA,5113,Tag1,Staatspolitik
2,20220005,1,Geschäft des Bundesrates,VBS,5113,Tag1,Sicherheitspolitik
3,20220006,1,Geschäft des Bundesrates,BK,5113,Tag1,Parlament
4,20220007,1,Geschäft des Bundesrates,EFD,5113,Tag1,Finanzwesen
...,...,...,...,...,...,...,...
11800,20227831,14,Fragestunde. Frage,EJPD,5116,Tag5,None
11801,20227832,14,Fragestunde. Frage,WBF,5116,Tag5,None
11802,20227833,14,Fragestunde. Frage,WBF,5116,Tag5,None
11803,20227834,14,Fragestunde. Frage,UVEK,5116,Tag5,None


In [48]:
# Group tags in category and map to a new column in business table

tagcat_list = {'value': ['Sicherheitspolitik', 'Staatspolitik', 'Parlament', 'Finanzwesen', 'Internationale Politik', 'Soziale Fragen', 'Umwelt', 'Wirtschaft', 'Verkehr', 'Recht Allgemein', 'Zivilrecht', 'Gesundheit', 'Wissenschaft und Forschung', 'Europapolitik', 'Medien und Kommunikation', 'Beschäftigung und Arbeit', 'Energie', 'Landwirtschaft', 'Bildung', 'Sozialer Schutz', 'Menschenrechte', 'Strafrecht', 'Migration', 'Gerichtswesen', 'Steuer', 'Raumplanung und Wohnungswesen', 'Kultur', 'Internationales Recht'],
        'TagCat': ['Politik', 'Politik', 'Politik', 'Finanz/Wirtschaft', 'Politik', 'Sozial', 'Energie/Umwelt/Land', 'Finanz/Wirtschaft', 'Energie/Umwelt/Land', 'Recht', 'Recht', 'Sozial', 'Bildung/Wissenschaft', 'Politik', 'Medien/Kultur', 'Sozial', 'Energie/Umwelt/Land', 'Energie/Umwelt/Land', 'Finanz/Wirtschaft', 'Sozial', 'Recht', 'Recht', 'Sozial', 'Recht', 'Finanz/Wirtschaft', 'Energie/Umwelt/Land', 'Medien/Kultur', 'Recht']}
df_TagCat = pd.DataFrame(tagcat_list)
df_TagCat

,value,TagCat
0,Sicherheitspolitik,Politik
1,Staatspolitik,Politik
2,Parlament,Politik
3,Finanzwesen,Finanz/Wirtschaft
4,Internationale Politik,Politik
5,Soziale Fragen,Sozial
6,Umwelt,Energie/Umwelt/Land
7,Wirtschaft,Finanz/Wirtschaft
8,Verkehr,Energie/Umwelt/Land
9,Recht Allgemein,Recht


In [49]:
# Add TagCat column

business_dfx3 =  pd.merge(business_dfx2, df_TagCat, on='value', how='left')
#business_dfx3.fillna('', inplace=True)
business_dfx3

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,variable,value,TagCat
0,20210081,1,Geschäft des Bundesrates,VBS,5113,Tag1,Sicherheitspolitik,Politik
1,20220001,1,Geschäft des Bundesrates,EDA,5113,Tag1,Staatspolitik,Politik
2,20220005,1,Geschäft des Bundesrates,VBS,5113,Tag1,Sicherheitspolitik,Politik
3,20220006,1,Geschäft des Bundesrates,BK,5113,Tag1,Parlament,Politik
4,20220007,1,Geschäft des Bundesrates,EFD,5113,Tag1,Finanzwesen,Finanz/Wirtschaft
...,...,...,...,...,...,...,...,...
11800,20227831,14,Fragestunde. Frage,EJPD,5116,Tag5,None,NaN
11801,20227832,14,Fragestunde. Frage,WBF,5116,Tag5,None,NaN
11802,20227833,14,Fragestunde. Frage,WBF,5116,Tag5,None,NaN
11803,20227834,14,Fragestunde. Frage,UVEK,5116,Tag5,None,NaN


In [50]:
# Cleaning the table

del business_dfx3["value"]  #delete column value

In [51]:
del business_dfx3["variable"]  #delete column variable

In [52]:
# delete incomplete data and duplicate
business_dfx3 = business_dfx3.dropna()
business_dfx3 = business_dfx3.drop_duplicates()
business_dfx3

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,TagCat
0,20210081,1,Geschäft des Bundesrates,VBS,5113,Politik
1,20220001,1,Geschäft des Bundesrates,EDA,5113,Politik
2,20220005,1,Geschäft des Bundesrates,VBS,5113,Politik
3,20220006,1,Geschäft des Bundesrates,BK,5113,Politik
4,20220007,1,Geschäft des Bundesrates,EFD,5113,Finanz/Wirtschaft
...,...,...,...,...,...,...
11457,20224182,5,Motion,EDA,5116,Sozial
11490,20224215,5,Motion,EJPD,5116,Sozial
11535,20227566,14,Fragestunde. Frage,VBS,5116,Recht
11616,20227647,14,Fragestunde. Frage,WBF,5116,Energie/Umwelt/Land


In [53]:
# Insert a column with value 1. This value is necessary in order to pivot the TagCat again
business_dfx3.insert(6, 'Val', '1')

In [54]:
business_dfx3

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,TagCat,Val
0,20210081,1,Geschäft des Bundesrates,VBS,5113,Politik,1
1,20220001,1,Geschäft des Bundesrates,EDA,5113,Politik,1
2,20220005,1,Geschäft des Bundesrates,VBS,5113,Politik,1
3,20220006,1,Geschäft des Bundesrates,BK,5113,Politik,1
4,20220007,1,Geschäft des Bundesrates,EFD,5113,Finanz/Wirtschaft,1
...,...,...,...,...,...,...,...
11457,20224182,5,Motion,EDA,5116,Sozial,1
11490,20224215,5,Motion,EJPD,5116,Sozial,1
11535,20227566,14,Fragestunde. Frage,VBS,5116,Recht,1
11616,20227647,14,Fragestunde. Frage,WBF,5116,Energie/Umwelt/Land,1


In [55]:
# Pivot the TagCat
business_dfx4 = business_dfx3.pivot_table('Val', ['ID', 'BusinessType', 'BusinessTypeName', 'ResponsibleDepartmentAbbreviation', 'SubmissionSession'], 'TagCat')

In [56]:
business_dfx4

,,,,TagCat,Bildung/Wissenschaft,Energie/Umwelt/Land,Finanz/Wirtschaft,Medien/Kultur,Politik,Recht,Sozial
ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,,,,,,,
20210081,1,Geschäft des Bundesrates,VBS,5113,NaN,NaN,NaN,NaN,1.0,NaN,1.0
20220001,1,Geschäft des Bundesrates,EDA,5113,NaN,NaN,NaN,NaN,1.0,NaN,NaN
20220003,1,Geschäft des Bundesrates,EFD,5114,NaN,NaN,1.0,NaN,NaN,NaN,NaN
20220005,1,Geschäft des Bundesrates,VBS,5113,NaN,NaN,NaN,NaN,1.0,NaN,NaN
20220006,1,Geschäft des Bundesrates,BK,5113,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20227833,14,Fragestunde. Frage,WBF,5116,NaN,1.0,NaN,NaN,1.0,NaN,NaN
20227834,14,Fragestunde. Frage,UVEK,5116,NaN,1.0,1.0,NaN,NaN,NaN,NaN
20227835,14,Fragestunde. Frage,WBF,5116,NaN,1.0,1.0,NaN,NaN,NaN,NaN


In [57]:
business_dfx4 = pd.DataFrame(business_dfx4.to_records())  # flatten the pivot table

In [58]:
business_dfx4.head() # preview

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,Bildung/Wissenschaft,Energie/Umwelt/Land,Finanz/Wirtschaft,Medien/Kultur,Politik,Recht,Sozial
0,20210081,1,Geschäft des Bundesrates,VBS,5113,NaN,NaN,NaN,NaN,1.0,NaN,1.0
1,20220001,1,Geschäft des Bundesrates,EDA,5113,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,20220003,1,Geschäft des Bundesrates,EFD,5114,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,20220005,1,Geschäft des Bundesrates,VBS,5113,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,20220006,1,Geschäft des Bundesrates,BK,5113,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [59]:
# Replace NaN with 0
business_dfx4 = business_dfx4.fillna(0)
 
# Finally, delete rows where ResponsibleDepartmentAbbreviation is empty  
business_dfx4['ResponsibleDepartmentAbbreviation'].replace('', np.nan, inplace=True)
business_dfx4.dropna(subset=['ResponsibleDepartmentAbbreviation'], inplace=True)
business_dfx4

,ID,BusinessType,BusinessTypeName,ResponsibleDepartmentAbbreviation,SubmissionSession,Bildung/Wissenschaft,Energie/Umwelt/Land,Finanz/Wirtschaft,Medien/Kultur,Politik,Recht,Sozial
0,20210081,1,Geschäft des Bundesrates,VBS,5113,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,20220001,1,Geschäft des Bundesrates,EDA,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,20220003,1,Geschäft des Bundesrates,EFD,5114,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,20220005,1,Geschäft des Bundesrates,VBS,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,20220006,1,Geschäft des Bundesrates,BK,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2300,20227832,14,Fragestunde. Frage,WBF,5116,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2301,20227833,14,Fragestunde. Frage,WBF,5116,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2302,20227834,14,Fragestunde. Frage,UVEK,5116,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2303,20227835,14,Fragestunde. Frage,WBF,5116,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [60]:
# 
# Now repeating the pivoting process with the ResponsibleDepartmentAbbreviation
#

In [61]:
# Insert a column with value 1. This value is necessary in order to pivot the ResponsibleDepartmentAbbreviation 
business_dfx4.insert(3, 'Val', '1')

In [62]:
business_dfx4

,ID,BusinessType,BusinessTypeName,Val,ResponsibleDepartmentAbbreviation,SubmissionSession,Bildung/Wissenschaft,Energie/Umwelt/Land,Finanz/Wirtschaft,Medien/Kultur,Politik,Recht,Sozial
0,20210081,1,Geschäft des Bundesrates,1,VBS,5113,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,20220001,1,Geschäft des Bundesrates,1,EDA,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,20220003,1,Geschäft des Bundesrates,1,EFD,5114,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,20220005,1,Geschäft des Bundesrates,1,VBS,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,20220006,1,Geschäft des Bundesrates,1,BK,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,20227832,14,Fragestunde. Frage,1,WBF,5116,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2301,20227833,14,Fragestunde. Frage,1,WBF,5116,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2302,20227834,14,Fragestunde. Frage,1,UVEK,5116,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2303,20227835,14,Fragestunde. Frage,1,WBF,5116,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [63]:
business_dpt = business_dfx4.pivot_table('Val', ['ID', 'BusinessType', 'BusinessTypeName', 'SubmissionSession', 'Bildung/Wissenschaft', 'Energie/Umwelt/Land', 'Finanz/Wirtschaft', 'Medien/Kultur', 'Politik', 'Recht', 'Sozial'], 'ResponsibleDepartmentAbbreviation')

In [64]:
business_dpt

,,,,,,,,,,ResponsibleDepartmentAbbreviation,BGer,BK,EDA,EDI,EFD,EJPD,Parl,UVEK,VBS,WBF
ID,BusinessType,BusinessTypeName,SubmissionSession,Bildung/Wissenschaft,Energie/Umwelt/Land,Finanz/Wirtschaft,Medien/Kultur,Politik,Recht,Sozial,,,,,,,,,,
20210081,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
20220001,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20220003,1,Geschäft des Bundesrates,5114,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
20220005,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
20220006,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20227832,14,Fragestunde. Frage,5116,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
20227833,14,Fragestunde. Frage,5116,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
20227834,14,Fragestunde. Frage,5116,0.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [65]:
business_dpt = pd.DataFrame(business_dpt.to_records())  # flatten the pivot table
business_dpt

,ID,BusinessType,BusinessTypeName,SubmissionSession,Bildung/Wissenschaft,Energie/Umwelt/Land,Finanz/Wirtschaft,Medien/Kultur,Politik,Recht,...,BGer,BK,EDA,EDI,EFD,EJPD,Parl,UVEK,VBS,WBF
0,20210081,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1,20220001,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220003,1,Geschäft des Bundesrates,5114,0.0,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,20220005,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,20220006,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,20227832,14,Fragestunde. Frage,5116,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2294,20227833,14,Fragestunde. Frage,5116,0.0,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2295,20227834,14,Fragestunde. Frage,5116,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2296,20227835,14,Fragestunde. Frage,5116,0.0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [66]:
# Finally, replace NaN with 0
business_dpt = business_dpt.fillna(0)
business_final = business_dpt
business_final

,ID,BusinessType,BusinessTypeName,SubmissionSession,Bildung/Wissenschaft,Energie/Umwelt/Land,Finanz/Wirtschaft,Medien/Kultur,Politik,Recht,...,BGer,BK,EDA,EDI,EFD,EJPD,Parl,UVEK,VBS,WBF
0,20210081,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20220001,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20220003,1,Geschäft des Bundesrates,5114,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,20220005,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,20220006,1,Geschäft des Bundesrates,5113,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,20227832,14,Fragestunde. Frage,5116,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2294,20227833,14,Fragestunde. Frage,5116,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2295,20227834,14,Fragestunde. Frage,5116,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2296,20227835,14,Fragestunde. Frage,5116,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [67]:
#business_final.to_csv('business_final.csv', encoding="utf-8-sig") #generate backup csv

# Merge Tables

In [68]:
# Merge Council members and Voting Tables
df_merge1 =  pd.merge(df_members_final, voting_final, on='PersonNumber', how='inner') 
df_merge1


,PersonNumber,FirstName,LastName,GenderAsString,CantonAbbreviation,PartyName,ParlGroupName,ParlGroupAbbreviation,BirthPlace_Canton,DateJoining,...,DateOfBirth,Age,Region_x,Region_y,IdVote,Decision,DecisionText,BusinessNumber,BusinessTitle,IdSession
0,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,1964-06-03 00:00:00+00:00,58,CH-de,Right,28693,1,Ja,20220003,Staatsrechnung 2021,5115
1,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,1964-06-03 00:00:00+00:00,58,CH-de,Right,28694,1,Ja,20220003,Staatsrechnung 2021,5115
2,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,1964-06-03 00:00:00+00:00,58,CH-de,Right,28695,1,Ja,20220003,Staatsrechnung 2021,5115
3,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,1964-06-03 00:00:00+00:00,58,CH-de,Right,28696,1,Ja,20220003,Staatsrechnung 2021,5115
4,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,1964-06-03 00:00:00+00:00,58,CH-de,Right,28697,1,Ja,20220007,Voranschlag 2022. Nachtrag I,5115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178883,4292,Simon,Stadler,m,UR,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,UR,2019-12-02 00:00:00+00:00,...,1988-05-02 00:00:00+00:00,34,CH-de,Center,28679,2,Nein,20203394,Stärkung der Ausbildung von Ärztinnen und Ärzt...,5114
178884,4292,Simon,Stadler,m,UR,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,UR,2019-12-02 00:00:00+00:00,...,1988-05-02 00:00:00+00:00,34,CH-de,Center,28680,2,Nein,20203499,Erarbeitung eines Aktionsplans Social Economy,5114
178885,4292,Simon,Stadler,m,UR,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,UR,2019-12-02 00:00:00+00:00,...,1988-05-02 00:00:00+00:00,34,CH-de,Center,28681,2,Nein,20203508,Stärkung der Milizpolitik. Anrechnung der Täti...,5114
178886,4292,Simon,Stadler,m,UR,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,UR,2019-12-02 00:00:00+00:00,...,1988-05-02 00:00:00+00:00,34,CH-de,Center,28682,2,Nein,20203529,Telearbeit. Chancen und Schutz für Arbeitnehme...,5114


In [69]:
#Add Table Business
#Business data without macthing Voting data are deleted with merging. There are however many voting data without business data. Only 50'000 rows complete out of 190'000
df_merge2 =  pd.merge(df_merge1, business_final, left_on='BusinessNumber', right_on = 'ID', how='inner')  #left when/if business data complete
print (df_merge2.shape)
M3_dataset = df_merge2


(49871, 42)


In [70]:
M3_dataset

,PersonNumber,FirstName,LastName,GenderAsString,CantonAbbreviation,PartyName,ParlGroupName,ParlGroupAbbreviation,BirthPlace_Canton,DateJoining,...,BGer,BK,EDA,EDI,EFD,EJPD,Parl,UVEK,VBS,WBF
0,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1138,Martin,Bäumle,m,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,3897,Tiana Angelina,Moser,f,ZH,Grünliberale Partei,Grünliberale Fraktion,GL,ZH,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49866,4008,Martin,Landolt,m,GL,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,GL,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
49867,4070,Martin,Candinas,m,GR,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,GR,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
49868,4282,Thomas,Rechsteiner,m,AI,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,AI,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
49869,4087,Jean-Paul,Gschwind,m,JU,Die Mitte,Die Mitte-Fraktion. Die Mitte. EVP.,M-E,JU,2019-12-02 00:00:00+00:00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [71]:
M3_dataset.to_csv('M3_dataset.csv', encoding="utf-8-sig") #generate FINAL csv file !

In [92]:
#Export csv to Drive
#!cp M3_dataset.csv "drive/My Drive/"